In [2]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import json
from collections import Counter

In [ ]:
# recipe_df = pd.read_csv("recipes_data.csv", nrows=1000)
# recipe_df.to_csv("recipes_1000.csv", index=False)

recipe_df = pd.read_csv("recipes_1000.csv")

In [3]:
recipe_df

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com
...,...,...,...,...,...,...,...
95,Baked Beans,"[""3 (1 lb.) cans pork and beans"", ""1/2 c. bell...","[""Cook onions and bell pepper in oil until oni...",www.cookbooks.com/Recipe-Details.aspx?id=775763,Gathered,"[""oil"", ""bell pepper"", ""black pepper"", ""brown ...",www.cookbooks.com
96,Sweet-N-Sour Chicken,"[""2 c. diced cooked chicken"", ""2 Tbsp. shorten...","[""Saute onion in 2 tablespoons shortening."", ""...",www.cookbooks.com/Recipe-Details.aspx?id=228506,Gathered,"[""vinegar"", ""ginger"", ""soy sauce"", ""shortening...",www.cookbooks.com
97,Ambrosia,"[""8 to 10 juicy oranges, peeled and diced"", ""1...","[""Combine all ingredients. Chill overnight."", ...",www.cookbooks.com/Recipe-Details.aspx?id=342478,Gathered,"[""sugar"", ""pecans"", ""moist coconut"", ""cherries...",www.cookbooks.com
98,Crazy Peanut Butter Cookies,"[""1 c. creamy peanut butter"", ""1 c. sugar"", ""1...","[""Mix together and roll into balls; flatten wi...",www.cookbooks.com/Recipe-Details.aspx?id=892363,Gathered,"[""egg"", ""peanut butter"", ""sugar""]",www.cookbooks.com


In [3]:
recipe_df = recipe_df.drop(columns=["source", "site"])
recipe_df

,title,ingredients,directions,link,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,"[""bite size shredded rice biscuits"", ""vanilla""..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,"[""cream of mushroom soup"", ""beef"", ""sour cream..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,"[""chicken gravy"", ""cream of mushroom soup"", ""c..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,"[""graham cracker crumbs"", ""powdered sugar"", ""p..."
...,...,...,...,...,...
95,Baked Beans,"[""3 (1 lb.) cans pork and beans"", ""1/2 c. bell...","[""Cook onions and bell pepper in oil until oni...",www.cookbooks.com/Recipe-Details.aspx?id=775763,"[""oil"", ""bell pepper"", ""black pepper"", ""brown ..."
96,Sweet-N-Sour Chicken,"[""2 c. diced cooked chicken"", ""2 Tbsp. shorten...","[""Saute onion in 2 tablespoons shortening."", ""...",www.cookbooks.com/Recipe-Details.aspx?id=228506,"[""vinegar"", ""ginger"", ""soy sauce"", ""shortening..."
97,Ambrosia,"[""8 to 10 juicy oranges, peeled and diced"", ""1...","[""Combine all ingredients. Chill overnight."", ...",www.cookbooks.com/Recipe-Details.aspx?id=342478,"[""sugar"", ""pecans"", ""moist coconut"", ""cherries..."
98,Crazy Peanut Butter Cookies,"[""1 c. creamy peanut butter"", ""1 c. sugar"", ""1...","[""Mix together and roll into balls; flatten wi...",www.cookbooks.com/Recipe-Details.aspx?id=892363,"[""egg"", ""peanut butter"", ""sugar""]"


In [4]:
# Fungsi untuk bersihkan dan konversi string list menjadi list Python asli
def clean_and_parse_json_list(x):
    if isinstance(x, str):
        # Ganti double double-quote menjadi satu double-quote
        x = x.replace('""', '"')

        # Kadang ada tanda kutip tunggal yang harus diganti ke double quotes
        # Untuk data kamu, coba replace tanda kutip tunggal di awal dan akhir
        if x.startswith("'") and x.endswith("'"):
            x = x[1:-1]

        # Pastikan tanda kutip didalam string di-escape dengan benar
        try:
            return json.loads(x)
        except json.JSONDecodeError:
            # Kalau gagal, coba raw string fix (misalnya escape manual)
            x = x.replace('\\"', '"').replace('\\\'', '\'')
            try:
                return json.loads(x)
            except Exception as e:
                print("Gagal parsing:", e)
                # fallback return string asli kalau error parsing
                return x
    return x

# Terapkan hanya ke kolom yang berbentuk list dalam string
for col in ["ingredients", "directions", "NER"]:
    recipe_df[col] = recipe_df[col].apply(clean_and_parse_json_list)

# Contoh cek hasil parsing kolom NER baris pertama
print(recipe_df["NER"].iloc[0])

# Kalau mau convert kolom NER jadi string (optional, misal untuk model input string)
recipe_df["NER_str"] = recipe_df["NER"].apply(lambda x: ", ".join(x) if isinstance(x, list) else "")

# Cek hasilnya
print(recipe_df["NER_str"].iloc[0])

Gagal parsing: Expecting ',' delimiter: line 1 column 22 (char 21)
['bite size shredded rice biscuits', 'vanilla', 'brown sugar', 'nuts', 'milk', 'butter']
bite size shredded rice biscuits, vanilla, brown sugar, nuts, milk, butter


In [27]:
# Buat user dan resep
recipes = recipe_df["title"].unique()
num_users = 5000
user_ids = [f"user_{i+1}" for i in range(num_users)]

# Bangun interactions
interactions = []
np.random.seed(42)

for user in user_ids:
    liked = np.random.choice(recipes, size=50, replace=False)
    for recipe in liked:
        interactions.append({"user_id": user, "title": recipe})

interaction_df = pd.DataFrame(interactions)

# Atur ulang seed untuk rating
np.random.seed(42)

# Jumlah total interaksi
n = len(interaction_df)

# Buat 80% rating > 3 (yaitu: 4 atau 5)
high_ratings = np.random.randint(4, 6, size=int(n * 0.9))  # 4 or 5
# Buat 20% rating <= 3 (1, 2, atau 3)
low_ratings = np.random.randint(1, 4, size=n - len(high_ratings))

# Gabungkan dan acak
all_ratings = np.concatenate([high_ratings, low_ratings])
np.random.shuffle(all_ratings)

# Masukkan ke DataFrame
interaction_df["rating"] = all_ratings

# Tampilkan tabel
interaction_df

,user_id,title,rating
0,user_1,Brown Rice,4
1,user_1,Three Bean Salad,4
2,user_1,Dot'S Civil War Cake,5
3,user_1,Chocolate Mint Bars,4
4,user_1,Fruit Pizza,4
...,...,...,...
249995,user_5000,Taco Dip,4
249996,user_5000,Sausage Balls,2
249997,user_5000,Chicken Roll-Ups,4
249998,user_5000,Cheeseburger Potato Soup,4


In [29]:
# ====================
# 1. Data Preparation
# ====================
interaction_df["user_id"] = interaction_df["user_id"].astype(str)
interaction_df["title"] = interaction_df["title"].astype(str)

unique_user_ids = interaction_df["user_id"].unique()
unique_titles = interaction_df["title"].unique()

train_df, test_df = train_test_split(interaction_df, test_size=0.2, random_state=42)

train_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": train_df["user_id"].values,
    "title": train_df["title"].values,
    "rating": train_df["rating"].values
}).shuffle(100).batch(768).cache()

val_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": test_df["user_id"].values,
    "title": test_df["title"].values,
    "rating": test_df["rating"].values,
}).batch(512)

recipe_dataset = tf.data.Dataset.from_tensor_slices(unique_titles)

# ====================
# 2. Lookup Layers
# ====================
user_lookup = tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None)
title_lookup = tf.keras.layers.StringLookup(vocabulary=unique_titles, mask_token=None)

# ====================
# 3. User & Item Model
# ====================
embedding_dim = 512

class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.user_embedding = tf.keras.Sequential([
            user_lookup,
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dim),
            tf.keras.layers.Dense(1024, activation="relu"),
            tf.keras.layers.Dense(embedding_dim)
        ])
    def call(self, inputs):
        return self.user_embedding(inputs)

class ItemModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.item_embedding = tf.keras.Sequential([
            title_lookup,
            tf.keras.layers.Embedding(len(unique_titles) + 1, embedding_dim),
            tf.keras.layers.Dense(1024, activation="relu"),
            tf.keras.layers.Dense(embedding_dim)
        ])
    def call(self, inputs):
        return self.item_embedding(inputs)

# ====================
# 4. Combined Model with Metrics
# ====================
class RecipeModel(tfrs.Model):
    def __init__(self, user_model, item_model):
        super().__init__()
        self.user_model = user_model
        self.item_model = item_model
        self.retrieval_task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=recipe_dataset.batch(128).map(lambda x: (x, item_model(x)))
            )
        )
        self.rating_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        item_embeddings = self.item_model(features["title"])

        rating_predictions = tf.reduce_sum(user_embeddings * item_embeddings, axis=1)

        rating_loss = self.rating_task(
            labels=features["rating"],
            predictions=rating_predictions
        )
        retrieval_loss = self.retrieval_task(user_embeddings, item_embeddings)

        return rating_loss + retrieval_loss

# ====================
# 5. Training
# ====================
user_model = UserModel()
item_model = ItemModel()
model = RecipeModel(user_model, item_model)

model.compile(optimizer=tf.keras.optimizers.Adam(0.0005))

early_stop = EarlyStopping(
    monitor="val_loss", 
    patience=5, 
    restore_best_weights=True
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[early_stop]
)

# ====================
# 6. Build Recommendation Index
# ====================
def map_title_to_embedding(title):
    embedding = item_model(tf.expand_dims(title, 0))
    return tf.expand_dims(title, 0), embedding

indexed_dataset = recipe_dataset.map(map_title_to_embedding)

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(indexed_dataset)


Epoch 1/100
261/261 [==============================] - 25s 92ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0098 - factorized_top_k/top_5_categorical_accuracy: 0.0450 - factorized_top_k/top_10_categorical_accuracy: 0.0843 - factorized_top_k/top_50_categorical_accuracy: 0.3977 - factorized_top_k/top_100_categorical_accuracy: 0.9120 - root_mean_squared_error: 1.6115 - loss: 5082.1621 - regularization_loss: 0.0000e+00 - total_loss: 5082.1621 - val_factorized_top_k/top_1_categorical_accuracy: 0.0044 - val_factorized_top_k/top_5_categorical_accuracy: 0.0392 - val_factorized_top_k/top_10_categorical_accuracy: 0.0842 - val_factorized_top_k/top_50_categorical_accuracy: 0.4728 - val_factorized_top_k/top_100_categorical_accuracy: 0.9996 - val_root_mean_squared_error: 0.9498 - val_loss: 1955.7428 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1955.7428
Epoch 2/100
261/261 [==============================] - 23s 89ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0065 - fa

In [35]:
# ====================
# 2. TF Dataset
# ====================
interaction_df["user_id"] = interaction_df["user_id"].astype(str)
interaction_df["title"] = interaction_df["title"].astype(str)

unique_user_ids = interaction_df["user_id"].unique()
unique_titles = interaction_df["title"].unique()

train_df, test_df = train_test_split(interaction_df, test_size=0.2, random_state=42)

train_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": train_df["user_id"],
    "title": train_df["title"]
}).shuffle(100).batch(768).cache()

val_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": test_df["user_id"].values,
    "title": test_df["title"].values,
    "rating": test_df["rating"].values,
}).batch(512)

recipe_dataset = tf.data.Dataset.from_tensor_slices(unique_titles)

# ====================
# 3. Lookup Layers
# ====================
user_lookup = tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None)
title_lookup = tf.keras.layers.StringLookup(vocabulary=unique_titles, mask_token=None)

# ====================
# 4. User & Item Model dengan Dense tambahan
# ====================
embedding_dim = 512

def diversity_loss(embeddings):
    normed = tf.math.l2_normalize(embeddings, axis=1)
    cosine_sim = tf.matmul(normed, normed, transpose_b=True)
    identity = tf.eye(tf.shape(cosine_sim)[0])
    diff = cosine_sim - identity
    return tf.reduce_mean(tf.abs(diff))

class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.user_embedding = tf.keras.Sequential([
            user_lookup,
            tf.keras.layers.Embedding(
                input_dim=len(unique_user_ids) + 1,
                output_dim=embedding_dim,
                activity_regularizer=tf.keras.regularizers.l2(1e-6)
            ),
            tf.keras.layers.Dense(1024, activation="relu"),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(embedding_dim)
        ])

    def call(self, inputs):
        return self.user_embedding(inputs)

class ItemModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.item_embedding = tf.keras.Sequential([
            title_lookup,
            tf.keras.layers.Embedding(
                input_dim=len(unique_titles) + 1,
                output_dim=embedding_dim,
                activity_regularizer=tf.keras.regularizers.l2(1e-6)
            ),
            tf.keras.layers.Dense(1024, activation="relu"),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(embedding_dim)
        ])

    def call(self, inputs):
        return self.item_embedding(inputs)

class RecipeModel(tfrs.Model):
    def __init__(self, user_model, item_model):
        super().__init__()
        self.user_model = user_model
        self.item_model = item_model
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=recipe_dataset.batch(128).map(lambda x: (x, item_model(x)))
            )
        )

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        item_embeddings = self.item_model(features["title"])
        main_loss = self.task(user_embeddings, item_embeddings)
        div_loss = diversity_loss(item_embeddings)
        return main_loss + 0.001 * div_loss

user_model = UserModel()
item_model = ItemModel()
model = RecipeModel(user_model, item_model)
model.compile(optimizer=tf.keras.optimizers.Adam(0.0005))

early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[early_stop])

# Bangun ulang index dengan model yang baru
indexed_dataset = recipe_dataset.map(lambda title: (tf.expand_dims(title, 0), item_model(tf.expand_dims(title, 0))))
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(indexed_dataset)


Epoch 1/100
261/261 [==============================] - 29s 103ms/step - factorized_top_k/top_1_categorical_accuracy: 0.4979 - factorized_top_k/top_5_categorical_accuracy: 0.5788 - factorized_top_k/top_10_categorical_accuracy: 0.6347 - factorized_top_k/top_50_categorical_accuracy: 0.8635 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 5078.4383 - regularization_loss: 8.3045e-07 - total_loss: 5078.4383 - val_factorized_top_k/top_1_categorical_accuracy: 0.0047 - val_factorized_top_k/top_5_categorical_accuracy: 0.0383 - val_factorized_top_k/top_10_categorical_accuracy: 0.0847 - val_factorized_top_k/top_50_categorical_accuracy: 0.4712 - val_factorized_top_k/top_100_categorical_accuracy: 0.9936 - val_loss: 1954.5615 - val_regularization_loss: 7.8559e-07 - val_total_loss: 1954.5615
Epoch 2/100
261/261 [==============================] - 39s 151ms/step - factorized_top_k/top_1_categorical_accuracy: 0.5440 - factorized_top_k/top_5_categorical_accuracy: 0.6405 - factorized_top_k/t

In [31]:
# ====================
# 9. Contoh Penggunaan
# ====================
user_id = "user_3"  # Pastikan ID dalam format string dan cocok dengan user_lookup
user_interactions = interaction_df[interaction_df["user_id"] == user_id]

print(f"Resep yang telah diberi rating oleh user {user_id}:")
for _, row in user_interactions.iterrows():
    print(f"- {row['title']} (Rating: {row['rating']})")

# Set judul yang sudah dikunjungi
visited_titles = set(user_interactions["title"].values)

# Ambil rekomendasi dari model
scores, titles = index(tf.constant([user_id]), k=50)

print(f"\nRekomendasi untuk user {user_id} (resep yang belum di-rate):")
count = 0
max_results = 20

for score, title in zip(scores[0].numpy(), titles[0].numpy()):
    title_str = title.decode("utf-8")
    if title_str in visited_titles:
        continue
    print(f"- {title_str} (Skor: {score:.4f})")
    count += 1
    if count >= max_results:
        break

if count == 0:
    print("Tidak ada rekomendasi resep baru untuk ditampilkan.")
elif count < max_results:
    print(f"(Hanya {count} resep baru yang bisa direkomendasikan)")


Resep yang telah diberi rating oleh user user_3:
- Mexican Cookie Rings (Rating: 5)
- Taco-Filled Green Pepper (Rating: 5)
- Cheese Dip (Rating: 5)
- Brown Rice (Rating: 4)
- Pink Stuff(Frozen Dessert)   (Rating: 5)
- Millionaire Pie (Rating: 4)
- Angel Biscuits (Rating: 5)
- Chocolate Frango Mints (Rating: 5)
- Sausage Balls (Rating: 4)
- Chicken Roll-Ups (Rating: 5)
- Fruit Pizza (Rating: 5)
- Corral Barbecued Beef Steak Strips (Rating: 4)
- Forever Amber (Rating: 4)
- Quick Barbecue Wings (Rating: 4)
- Grandma Hanrath'S Banana Breadfort Collins, Colorado   (Rating: 4)
- Beer Bread (Rating: 5)
- Chocolate Mint Bars (Rating: 5)
- Chicken Stew (Rating: 4)
- Taco Salad Chip Dip (Rating: 3)
- Gooey Coffee Cake (Rating: 5)
- Three Bean Salad (Rating: 5)
- Chicken Ole (Rating: 5)
- Potato Casserole (Rating: 5)
- Beef And Spanish Rice Casserole (Rating: 3)
- Crab Cakes (Rating: 4)
- Cheeseburger Potato Soup (Rating: 4)
- Chicken Funny (Rating: 4)
- Scalloped Corn (Rating: 2)
- Cranberry Sal

In [33]:
# Hitung popularitas item
item_popularity = Counter(interaction_df["title"])
total_interaction = sum(item_popularity.values())

# Set item unik untuk Coverage
all_items = set(interaction_df["title"].unique())

# Riwayat user untuk Serendipity
user_history = interaction_df.groupby("user_id")["title"].apply(set).to_dict()

# ====================
# Precision@K
# ====================
def precision_at_k(k):
    correct = 0
    total = 0
    test_dict = test_df.groupby("user_id")["title"].apply(set).to_dict()

    for user_id, true_items in test_dict.items():
        _, recommended = index(tf.constant([user_id]), k=k)
        recommended_ids = set(title.numpy().decode("utf-8") for title in recommended[0])
        hits = recommended_ids.intersection(true_items)
        correct += len(hits)
        total += k
    return correct / total if total > 0 else 0

# ====================
# Novelty@K
# ====================
def novelty_at_k(k):
    total_novelty = 0.0
    total_users = 0

    for user_id in test_df["user_id"].unique():
        _, recommended = index(tf.constant([user_id]), k=k)
        novelty_score = 0.0
        for title in recommended[0].numpy():
            title_str = title.decode("utf-8")
            freq = item_popularity[title_str] / total_interaction
            if freq > 0:
                novelty_score += -np.log2(freq)
        avg_novelty = novelty_score / k
        total_novelty += avg_novelty
        total_users += 1

    return total_novelty / total_users if total_users > 0 else 0

# ====================
# Coverage@K
# ====================
def coverage_at_k(k):
    recommended_items = set()
    for user_id in test_df["user_id"].unique():
        _, recommended = index(tf.constant([user_id]), k=k)
        for title in recommended[0].numpy():
            recommended_items.add(title.decode("utf-8"))
    return len(recommended_items) / len(all_items)

# ====================
# Serendipity@K
# ====================
def serendipity_at_k(k):
    serendipity_sum = 0.0
    user_count = 0

    for user_id in test_df["user_id"].unique():
        seen = user_history.get(user_id, set())
        seen_vecs = []

        for title in seen:
            vec = model.item_model(tf.constant([title])).numpy()
            seen_vecs.append(vec[0])

        if not seen_vecs:
            continue

        seen_avg = np.mean(seen_vecs, axis=0).reshape(1, -1)

        _, recommended = index(tf.constant([user_id]), k=k)
        ser = 0

        for title in recommended[0].numpy():
            title_str = title.decode("utf-8")
            if title_str in seen:
                continue
            rec_vec = model.item_model(tf.constant([title_str])).numpy().reshape(1, -1)
            sim = cosine_similarity(seen_avg, rec_vec)[0][0]
            ser += 1 - sim

        serendipity_sum += ser / k
        user_count += 1

    return serendipity_sum / user_count if user_count > 0 else 0

# ====================
# Evaluasi Semua Metrik
# ====================
def evaluate_model(k_values=[10]):
    print("Evaluasi sistem rekomendasi dengan model eksploratif:")
    for k in k_values:
        precision = precision_at_k(k)
        novelty = novelty_at_k(k)
        coverage = coverage_at_k(k)
        serendipity = serendipity_at_k(k)
        print(f"\nTop-{k}")
        print(f"- Precision@{k}:   {precision:.4f}")
        print(f"- Novelty@{k}:     {novelty:.4f}")
        print(f"- Coverage@{k}:    {coverage:.4f}")
        print(f"- Serendipity@{k}: {serendipity:.4f}")

# Jalankan evaluasi
evaluate_model(k_values=[10])

Evaluasi sistem rekomendasi dengan model eksploratif:

Top-10
- Precision@10:   0.0896
- Novelty@10:     6.6498
- Coverage@10:    0.6000
- Serendipity@10: 0.0061
